In [1]:
import pickle

from BertTM import *

unable to import 'smart_open.gcs', disabling that module


## BERT models

### Load pretrained

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', )
model = BertForSequenceClassificationOutputPooled.from_pretrained('bert-base-uncased', 
                                                              output_attentions=True, 
                                                              output_hidden_states=True)

### Load fine-tuned

In [2]:
output_dir = "../bert-classifier-pytorch/model_save_attention_1epoch"

# Load a trained model and vocabulary that you have fine-tuned
model = BertForSequenceClassificationOutputPooled.from_pretrained(output_dir,
                                                      output_attentions = True, 
                                                      output_hidden_states = True)
tokenizer = BertTokenizer.from_pretrained(output_dir)

### Test model

In [ ]:
input_list = []
token_list = []
cls_ = '[CLS]'
sep_ = '[SEP]'
sentences = ['Hello, my dog is cute and cutest.', 'I am too']
for i, sent in enumerate(sentences):
    inputs = tokenizer.encode_plus(sentences[i], add_special_tokens=True)
    tokens = [cls_] + tokenizer.tokenize(sentences[i]) + [sep_]
    input_ids = torch.tensor(inputs['input_ids']).unsqueeze(0)
    input_list.append(input_ids)
    token_list.append(tokens)

### Load sentence embedder

In [35]:
word_embedding_model = models.BERT(output_dir, max_seq_length = 240,)

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

st_model = SentenceTransformer(modules=[word_embedding_model, pooling_model],
                               #device=torch.device("cuda")
                              )

### Test that attention and vectorization work

In [111]:
attentions = get_attention(sentences, model, tokenizer, method = 'first')
np.sum([tpl[1] for tpl in attentions[1]])

vectorized = vectorize(sentences, model, tokenizer)
torch.stack(vectorized).detach().numpy().shape

(2, 768)

In [120]:
get_attention(["this movie was the cutest. read more at http://worstever.com"], model, tokenizer, method = 'first')

[[('this', 0.0613682),
  ('movie', 0.030429687),
  ('was', 0.035641603),
  ('the', 0.21911351),
  ('cutest', 0.06270852),
  ('.', 0.09440403),
  ('read', 0.0423442),
  ('more', 0.06480052),
  ('at', 0.06262489),
  ('http', 0.027938599),
  (':', 0.043154325),
  ('/', 0.038011733),
  ('/', 0.041409045),
  ('worstever', 0.04187157),
  ('.', 0.08471608),
  ('com', 0.049463503)]]

## Topic Model from BERT

### Load data and set params

In [3]:
df = pd.read_csv("nlwx_2020_hashtags_no_rt_predictions.csv")
data = df['text']
ngram = (1, 3)
n_topics = 9

### Create embeddings

In [40]:
%%time

rows, attentions = get_embeddings(data, model, tokenizer)

Processed 0 rows in 0.15 seconds.
Processed 500 rows in 98.27 seconds.
Processed 1000 rows in 193.77 seconds.
Processed 1500 rows in 290.81 seconds.
Processed 2000 rows in 386.74 seconds.
Processed 2500 rows in 484.5 seconds.
Processed 3000 rows in 584.24 seconds.
Processed 3500 rows in 679.84 seconds.
Processed 4000 rows in 776.88 seconds.
Processed 4500 rows in 873.04 seconds.
Processed 5000 rows in 969.55 seconds.
Processed 5500 rows in 1067.45 seconds.
Processed 6000 rows in 1168.92 seconds.
Processed 6500 rows in 1268.35 seconds.
Processed 7000 rows in 1366.79 seconds.
Processed 7500 rows in 1468.06 seconds.
Processed 8000 rows in 1570.23 seconds.
Processed 8500 rows in 1671.15 seconds.
Processed 9000 rows in 1772.01 seconds.
Processed 9500 rows in 1872.32 seconds.
Processed 10000 rows in 1970.01 seconds.
Processed 10500 rows in 2071.37 seconds.
Processed 11000 rows in 2172.99 seconds.
Processed 11500 rows in 2273.16 seconds.
Processed 12000 rows in 2370.85 seconds.
Processed 1250

Save data after creating embeddings

In [ ]:
all_model_data = []

for i in range(len(rows)):
    all_model_data.append((data[i], df.prediction[i], attentions[i], rows[i]))
    
#pickle.dump(all_model_data, open(f"attentions_sent_embeddings.pkl", "wb" ))

Define stopwords.

In [58]:
hashtags = ['nlwhiteout', 'nlweather', 'newfoundland', 'nlblizzard2020', 'nlstorm2020',
 'snowmaggedon2020', 'stormageddon2020', 'snowpocalypse2020', 'snowmageddon',
 'nlstorm', 'nltraffic', 'nlwx', 'nlblizzard']
def get_stopwords(hashtags = [], filename = 'stopwords-en.json'):
    with open(filename) as fopen:
        stopwords = json.load(fopen)

    stopwords.extend(['#', '@', '…', "'", "’", "[UNK]", "\"", ";", "*", "_", "amp", "&", "“", "”"] + hashtags)
    return(stopwords)


stopwords = get_stopwords()
print(len(stopwords))

1312


Load pickled embedding data.

In [59]:
all_model_data = pickle.load(open("attentions_sent_embeddings.pkl", "rb"))
texts, _, attentions, rows = zip(*all_model_data)

### Kmeans model

Train kmeans model.

In [60]:
%%time

labels = get_clusters(rows, n_topics)

Fitting kmeans model.
The number of texts per label are:
{0: 2632, 1: 4438, 2: 1034, 3: 826, 4: 670, 5: 798, 6: 2858, 7: 4467, 8: 4074}
CPU times: user 41.7 s, sys: 1.81 s, total: 43.5 s
Wall time: 37.9 s


#### Cluster components

Drop stopwords and empty documents.

In [61]:
%%time
  
filtered_a, filtered_t, filtered_l = filter_data(attentions, stopwords, labels)

Filtering attentions.
CPU times: user 13.8 s, sys: 4 ms, total: 13.8 s
Wall time: 13.8 s


Use Gensim's phraser to determine which ngram to include a word in.

In [62]:
%%time

features = get_phrases(filtered_t, min_count=10, threshold=0.5)
pickle.dump(features, open('features.pickle', 'wb'))

CPU times: user 5.28 s, sys: 8 ms, total: 5.29 s
Wall time: 5.32 s


Use BERT's attention mechanism to determine which words characterize each kmeans cluster.

In [63]:
%%time
    
components, words_label = determine_cluster_components(filtered_l, filtered_a, ngram, features)


Determining cluster components. This will take awhile. 
Progress will be printed for every 500th processed property.
    
Processed 5000 texts in 1.99 seconds.
Processed 10000 texts in 3.99 seconds.
Processed 15000 texts in 6.09 seconds.
Processed 20000 texts in 8.26 seconds.
Finished determining a total of 21797 cluster components. Total time 8.97 seconds.
CPU times: user 8.99 s, sys: 0 ns, total: 8.99 s
Wall time: 8.98 s


Use term frequency inverse cluster frequency with and without BERT's attentions mechanism to determine which words characterize each kmeans cluster. 

In [64]:
%%time

tfidf_indexed = tf_icf(words_label, n_topics)
components_tfidf, components_tfidf_attn = get_tfidf_components(components, tfidf_indexed)

CPU times: user 304 ms, sys: 0 ns, total: 304 ms
Wall time: 305 ms


#### Topics

In [65]:
topics_attn = topics_df(
    topics = n_topics,
    components = components,
    n_words = 10)

pickle.dump(topics_attn, open("topics_sent_embed_attn.pickle", "wb"))
topics_attn

,topic 0,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8
0,snow,newfoundland,power,assistance,hope,stay safe,snow,snow,newfoundland
1,storm,nlwx,closed,helping,hoping,safe,storm,stormageddon2020,snow
2,stormageddon2020,nlblizzard2020,power outage,support,thinking,warning,people,storm,storm
3,emergency,snow,road,people,stay safe,storm,power,newfoundland,nlwx
4,wind,snowmaggedon2020,snow,emergency,newfoundland,stay,car,snowmaggedon2020,shovel
5,newfoundland,nlstorm,storm,supply,prayer,newfoundland,digging,blizzard,snowmaggedon2020
6,blizzard,nlstorm2020,emergency,community,safe,emergency,missing,snowstorm,people
7,weather,nltraffic,lost power,food,god,envcanada advisory blowingsnow,stuck,weather,love
8,snowfall,snowmageddon,outage,newfoundland,storm,alert,emergency,winter,hope
9,update,snowstorm,damage,service,hope safe,blizzard warning,newfoundland,snowpocalypse2020,day


In [66]:
topics_tfidf = topics_df(
    topics = n_topics,
    components = components_tfidf,
    n_words = 10)

pickle.dump(topics_tfidf, open("topics_sent_embed_tfidf.pickle", "wb"))
topics_tfidf

,topic 0,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8
0,nlwx,nlwx,power,assistance,prayer,stay safe,nlwx,nlwx,nlwx
1,snow,nlblizzard2020,road,newfoundland,hope,nlwx,snow,snow,newfoundland
2,newfoundland,newfoundland,nlwx,people,newfoundland,newfoundland,car,snowmaggedon2020,snowmaggedon2020
3,storm,snowmaggedon2020,power outage,emergency,nlblizzard2020,safe,people,stormageddon2020,nlblizzard2020
4,wind,nltraffic,john,nlwx,stay safe,storm,house,newfoundland,snow
5,emergency,nlstorm2020,snow,helping,thinking,blizzard warning,john,storm,day
6,stormageddon2020,snowmageddon,emergency,support,safe,envcanada advisory blowingsnow,road,nlblizzard2020,time
7,john,snow,closed,community,hope safe,nlblizzard2020,door,blizzard,people
8,update,nlwx nltraffic,storm,food,nlwx,warning,street,snowpocalypse2020,love
9,blizzard,nlwx nlblizzard2020,newfoundland,john,hoping,emergency,newfoundland,snowmageddon,dog


In [67]:
topics_tfidf_attn = topics_df(
    topics = n_topics,
    components = components_tfidf_attn,
    n_words = 10)

pickle.dump(topics_tfidf_attn, open("topics_sent_embed_tfidf_attn.pickle", "wb"))
topics_tfidf_attn

,topic 0,topic 1,topic 2,topic 3,topic 4,topic 5,topic 6,topic 7,topic 8
0,snow,nlwx,power,assistance,hope,stay safe,snow,snow,newfoundland
1,storm,newfoundland,road,people,newfoundland,safe,people,stormageddon2020,nlwx
2,newfoundland,nlblizzard2020,closed,helping,prayer,newfoundland,car,snowmaggedon2020,snow
3,emergency,snowmaggedon2020,power outage,emergency,hoping,storm,storm,newfoundland,snowmaggedon2020
4,stormageddon2020,nltraffic,snow,support,thinking,nlwx,power,storm,people
5,wind,snow,emergency,newfoundland,stay safe,warning,nlwx,blizzard,love
6,blizzard,nlstorm2020,storm,community,safe,stay,newfoundland,nlwx,day
7,nlwx,snowmageddon,outage,food,hope safe,envcanada advisory blowingsnow,road,snowstorm,time
8,john,nlstorm,john,supply,storm,emergency,house,snowpocalypse2020,nlblizzard2020
9,update,snowstorm,lost power,service,friend,blizzard warning,door,winter,storm
